In [1]:
!git clone https://github.com/AlexeyAB/darknet/
%cd darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15379, done.
remote: Total 15379 (delta 0), reused 0 (delta 0), pack-reused 15379
Receiving objects: 100% (15379/15379), 14.01 MiB | 18.16 MiB/s, done.
Resolving deltas: 100% (10340/10340), done.
/content/darknet


In [2]:
!apt install libopencv-dev python-opencv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
The following NEW packages will be installed:
  python-opencv
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 535 kB of archives.
After this operation, 2,944 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [535 kB]
Fetched 535 kB in 1s (611 kB/s)
Selecting previously unselected package python-opencv.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../python-opencv_3.2.0+dfsg-4ubuntu0.1_amd64.deb ...
Unpacking python-opencv (3.2.0+dfsg-4ubuntu0.1) ...
Setting up python-opencv (3.2.0+dfsg-4ubuntu0.1) ...


In [3]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/g' Makefile

In [4]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/

In [5]:
%cp cfg/yolov3.cfg cfg/yolo-obj.cfg
!sed -i 's/batch=1/batch=64/g' cfg/yolo-obj.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/g' cfg/yolo-obj.cfg
!sed -i 's/steps=400000,450000/steps=4800,5400/g' cfg/yolo-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=16/g' cfg/yolo-obj.cfg
!sed -i 's/classes=80/classes=3/g' cfg/yolo-obj.cfg
!sed -i 's/filters=255/filters=24/g' cfg/yolo-obj.cfg

In [6]:
!touch /content/darknet/build/darknet/x64/data/obj.names
!echo "person" > /content/darknet/build/darknet/x64/data/obj.names
!echo "protective vest" >> /content/darknet/build/darknet/x64/data/obj.names
!echo "hard hat" >> /content/darknet/build/darknet/x64/data/obj.names

In [7]:
!touch /content/darknet/build/darknet/x64/data/obj.data
!echo "classes = 3" > /content/darknet/build/darknet/x64/data/obj.data
!echo "train  = build/darknet/x64/data/train.txt" >> /content/darknet/build/darknet/x64/data/obj.data
!echo "valid  = build/darknet/x64/data/test.txt" >> /content/darknet/build/darknet/x64/data/obj.data
!echo "names = build/darknet/x64/data/obj.names" >> /content/darknet/build/darknet/x64/data/obj.data
!echo "backup = backup/" >> /content/darknet/build/darknet/x64/data/obj.data

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd /content

/content


In [10]:
%cp -r drive/MyDrive/train/. darknet/build/darknet/x64/data/obj/

In [11]:
%ls -1 darknet/build/darknet/x64/data/obj/*.jpg | wc -l
%ls -1 darknet/build/darknet/x64/data/obj/*.txt | wc -l

3222
3222


In [12]:
%cp -r drive/MyDrive/data/train.txt darknet/build/darknet/x64/data/
%cp -r drive/MyDrive/data/test.txt darknet/build/darknet/x64/data/

In [13]:
!wget -P darknet/build/darknet/x64/ https://pjreddie.com/media/files/darknet53.conv.74

--2021-12-18 20:31:54--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet/build/darknet/x64/darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  55.3MB/s    in 2.8s    

2021-12-18 20:31:57 (55.3 MB/s) - ‘darknet/build/darknet/x64/darknet53.conv.74’ saved [162482580/162482580]



In [14]:
%cd darknet

/content/darknet


In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show -map

Выходные данные были обрезаны до нескольких последних строк (5000).
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.336231), count: 24, class_loss = 131.536804, iou_loss = 15.805756, total_loss = 147.342560 
 total_bbox = 47046, rewritten_bbox = 3.111848 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 66.974281, iou_loss = 0.000000, total_loss = 66.974281 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.261166), count: 1, class_loss = 110.456238, iou_loss = 0.628784, total_loss = 111.085022 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.258203), count: 41, class_loss = 138.643692, iou_loss = 41.623199, total_loss = 180.266891 
 total_bbox = 47088, rewritten_bbox = 3.113320 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 65.883766, iou_loss = 0.00

In [15]:
%cd /content
%cp drive/MyDrive/data/yolo-obj_4300.weights darknet/backup/yolo-obj_last.weights
%cd darknet

/content
/content/darknet


In [25]:
%cd /content
%cp darknet/backup/yolo-obj_last.weights drive/MyDrive/data/yolo-obj_4800.weights
%cd darknet

/content
/content/darknet


In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg backup/yolo-obj_last.weights -dont_show -map

Выходные данные были обрезаны до нескольких последних строк (5000).
 total_bbox = 776, rewritten_bbox = 1.804124 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.818828), count: 9, class_loss = 0.058928, iou_loss = 0.188598, total_loss = 0.247526 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.789177), count: 22, class_loss = 0.374999, iou_loss = 0.532516, total_loss = 0.907515 
 total_bbox = 807, rewritten_bbox = 1.734820 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.839674), count: 20, class_loss = 0.218772, iou_loss = 0.391099, total_loss = 0.6